In [1]:
import os


project_dir = '/blue/shenhaowang/qingqisong/be-and-active-travel'
os.chdir(project_dir)

In [2]:
"""
================================================================================
Chicago Travel Behavior - Multi-Model Analysis (Transit Focus)
================================================================================
Purpose: Comprehensive statistical and machine learning analysis of TRANSIT
         travel behavior determinants including built environment factors.

Dependent Variables:
    - Y_transit_duration_min (Continuous)
    - n_transit_trips (Count)
    - has_transit_travel (Binary)

Models:
    A. OLS Regression (Interpretability)
    B. Gradient Boosting (Non-linear)
    C. Random Forest (Robustness)
    D. Poisson Regression (Count data - n_transit_trips)
    E. Logistic Regression (Binary - has_transit_travel)


================================================================================
"""

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import PoissonRegressor, LogisticRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, roc_curve, confusion_matrix,
                             classification_report)
from sklearn.inspection import PartialDependenceDisplay

# Statsmodels imports
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.discrete.discrete_model import Logit, Poisson

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.size'] = 11
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
print("=" * 80)
print("    CHICAGO TRAVEL BEHAVIOR - TRANSIT ANALYSIS")
print("    芝加哥出行行为 - 公交出行分析")
print("=" * 80)

# ============================================================================
# CONFIGURATION
# ============================================================================

INPUT_FILE = './city_home_based_chicago_research_ready_v3_clean.csv'
OUTPUT_DIR = './city_home_based_model_results_transit'

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define variables
DEPENDENT_VARS = {
    'Y_transit_duration_min': 'continuous',
    'n_transit_trips': 'count',
    'has_transit_travel': 'binary'
}

DEMO_VARS = [
    'age', 'Female', 'Employed', 'Bachelor_Above', 
    'hhinc', 'hhveh', 'Home_Owner', 'White', 'Black', 'Hispanic'
]

BE_VARS = [
    'D1B', 'D1C', 'D2A_EPHHM', 
    'H_intersection_density', 'H_road_network_complexity', 'H_building_density',
    'dist_cbd_mi', 'dist_rail_mi', 'dist_park_mi', 'bus_count_14mile'
]

INDEPENDENT_VARS = DEMO_VARS + BE_VARS

print(f"\nConfiguration:")
print(f"  Dependent Variables:")
for var, var_type in DEPENDENT_VARS.items():
    print(f"    - {var} ({var_type})")
print(f"  Demographic Variables: {len(DEMO_VARS)}")
print(f"  Built Environment Variables: {len(BE_VARS)}")


# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================

print("\n" + "=" * 80)
print("[STEP 1] Loading Data / 加载数据")
print("=" * 80)

df = pd.read_csv(INPUT_FILE)
print(f"\n  Loaded: {len(df):,} observations")

# Select needed columns
all_vars = list(DEPENDENT_VARS.keys()) + INDEPENDENT_VARS
df_model = df[all_vars].copy().dropna()
print(f"  Final sample size: {len(df_model):,}")

# Check dependent variable distributions
print("\n  Dependent Variable Summary:")
for var, var_type in DEPENDENT_VARS.items():
    data = df_model[var]
    print(f"\n    {var} ({var_type}):")
    print(f"      Mean: {data.mean():.2f}, Std: {data.std():.2f}")
    print(f"      Min: {data.min():.0f}, Max: {data.max():.0f}")
    if var_type == 'binary':
        print(f"      Class 1: {data.sum():,} ({data.mean()*100:.1f}%)")
        print(f"      Class 0: {(1-data).sum():,} ({(1-data.mean())*100:.1f}%)")

In [2]:
# ============================================================================
# STEP 2: EXPLORATORY DATA ANALYSIS
# ============================================================================

print("\n" + "=" * 80)
print("[STEP 2] Exploratory Data Analysis / 探索性数据分析")
print("=" * 80)

# ============================================================================
# Visualization 1: Distribution of Dependent Variables
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Y_transit_duration_min - Histogram
ax = axes[0]
data = df_model['Y_transit_duration_min']
ax.hist(data[data > 0], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(data[data > 0].mean(), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {data[data > 0].mean():.1f}')
ax.set_xlabel('Minutes')
ax.set_ylabel('Frequency')
ax.set_title('Transit Duration (Non-Zero)\n公交出行时长分布')
ax.legend()

# n_transit_trips - Count distribution
ax = axes[1]
data = df_model['n_transit_trips']
counts = data.value_counts().sort_index()
ax.bar(counts.index[:15], counts.values[:15], color='coral', edgecolor='black')
ax.set_xlabel('Number of Trips')
ax.set_ylabel('Frequency')
ax.set_title('Transit Trip Count\n公交出行次数分布')

# has_transit_travel - Pie chart
ax = axes[2]
data = df_model['has_transit_travel']
sizes = [data.sum(), len(data) - data.sum()]
labels = ['Has Transit\n有公交出行', 'No Transit\n无公交出行']
colors = ['seagreen', 'lightgray']
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90,
       explode=(0.05, 0))
ax.set_title('Transit Travel Participation\n公交出行参与率')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/01_transit_dv_distributions.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n  ✓ Saved: 01_transit_dv_distributions.png")

# ============================================================================
# Visualization 2: Transit Use by Demographics
# ============================================================================

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

demo_binary = ['Female', 'Employed', 'Bachelor_Above', 'Home_Owner', 
               'White', 'Black', 'Hispanic', 'hhveh']

for idx, var in enumerate(demo_binary):
    row = idx // 4
    col = idx % 4
    ax = axes[row, col]
    
    if var == 'hhveh':
        # Group by vehicle count
        groups = df_model.groupby(var)['has_transit_travel'].mean() * 100
        ax.bar(groups.index, groups.values, color='steelblue', edgecolor='black')
        ax.set_xlabel('Vehicles')
    else:
        # Binary comparison
        means = df_model.groupby(var)['has_transit_travel'].mean() * 100
        ax.bar(['No', 'Yes'], means.values, color=['lightcoral', 'steelblue'], edgecolor='black')
    
    ax.set_ylabel('% Using Transit')
    ax.set_title(f'Transit Use by {var}')

plt.suptitle('Transit Travel Participation by Demographics\n人口特征与公交出行参与率', fontsize=14)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/02_transit_by_demographics.png', dpi=150, bbox_inches='tight')
plt.close()
print("  ✓ Saved: 02_transit_by_demographics.png")

# ============================================================================
# Visualization 3: Transit Use by Built Environment
# ============================================================================



# ============================================================================
# Visualization 3: Transit Use by Built Environment
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

be_key_vars = ['D1B', 'dist_cbd_mi', 'dist_rail_mi', 
               'bus_count_14mile', 'D2A_EPHHM', 'H_building_density']

for idx, var in enumerate(be_key_vars):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    try:
        # Try qcut with duplicates='drop'
        df_model['_quartile'] = pd.qcut(df_model[var], q=4, 
                                         labels=False, 
                                         duplicates='drop')
        
        # Create labels based on actual number of bins
        n_bins = df_model['_quartile'].nunique()
        
        if n_bins >= 2:
            # Transit participation by quartile
            means = df_model.groupby('_quartile')['has_transit_travel'].mean() * 100
            
            colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(means)))
            bars = ax.bar(range(len(means)), means.values, color=colors, edgecolor='black')
            ax.set_xticks(range(len(means)))
            ax.set_xticklabels([f'Q{i+1}' for i in range(len(means))])
            ax.set_ylabel('% Using Transit')
            ax.set_xlabel(var)
            ax.set_title(f'Transit Use by {var}')
        else:
            ax.text(0.5, 0.5, f'{var}\nInsufficient variation', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{var}')
            
    except Exception as e:
        # Fallback: use median split
        median_val = df_model[var].median()
        df_model['_group'] = (df_model[var] > median_val).astype(int)
        means = df_model.groupby('_group')['has_transit_travel'].mean() * 100
        
        ax.bar(['Below Median', 'Above Median'], means.values, 
               color=['lightblue', 'steelblue'], edgecolor='black')
        ax.set_ylabel('% Using Transit')
        ax.set_title(f'Transit Use by {var}')
        
        if '_group' in df_model.columns:
            df_model.drop('_group', axis=1, inplace=True)
    
    # Clean up
    if '_quartile' in df_model.columns:
        df_model.drop('_quartile', axis=1, inplace=True)

plt.suptitle('Transit Travel by Built Environment Characteristics\n建成环境特征与公交出行', fontsize=14)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/03_transit_by_be.png', dpi=150, bbox_inches='tight')
plt.close()
print("  ✓ Saved: 03_transit_by_be.png")

# ============================================================================
# Visualization 4: Correlation Matrix
# ============================================================================

fig, ax = plt.subplots(figsize=(16, 14))

corr_vars = list(DEPENDENT_VARS.keys()) + INDEPENDENT_VARS
corr_matrix = df_model[corr_vars].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5, ax=ax,
            annot_kws={'size': 7})

ax.set_title('Correlation Matrix\n相关系数矩阵', fontsize=14)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/04_correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.close()
print("  ✓ Saved: 04_correlation_matrix.png")

In [2]:
# ============================================================================
# STEP 3: MULTICOLLINEARITY CHECK (VIF)
# ============================================================================

print("\n" + "=" * 80)
print("[STEP 3] Multicollinearity Check (VIF) / 多重共线性检验")
print("=" * 80)

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data['Variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values('VIF', ascending=False)

X_vif = df_model[INDEPENDENT_VARS].copy()
vif_df = calculate_vif(X_vif)

print("\n  Variance Inflation Factors:")
print("  " + "-" * 50)

high_vif_vars = []
for _, row in vif_df.iterrows():
    flag = " ⚠ HIGH" if row['VIF'] > 7 else ""
    print(f"  {row['Variable']:<35} {row['VIF']:>10.2f}{flag}")
    if row['VIF'] > 7:
        high_vif_vars.append(row['Variable'])

vif_df.to_csv(f'{OUTPUT_DIR}/vif_results.csv', index=False)

# VIF Chart
fig, ax = plt.subplots(figsize=(12, 8))
colors = ['red' if v > 7 else 'steelblue' for v in vif_df['VIF']]
ax.barh(vif_df['Variable'], vif_df['VIF'], color=colors, edgecolor='black')
ax.axvline(x=7, color='red', linestyle='--', linewidth=2, label='VIF = 7 Threshold')
ax.set_xlabel('Variance Inflation Factor')
ax.set_title('Multicollinearity Check: VIF\n多重共线性检验', fontsize=14)
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/05_vif_chart.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n  ✓ Saved: 05_vif_chart.png")

In [2]:
# ============================================================================
# STEP 4: DATA PREPROCESSING
# ============================================================================

print("\n" + "=" * 80)
print("[STEP 4] Data Preprocessing / 数据预处理")
print("=" * 80)

binary_vars = ['Female', 'Employed', 'Bachelor_Above', 'Home_Owner', 'White', 'Black', 'Hispanic']
continuous_vars = [v for v in INDEPENDENT_VARS if v not in binary_vars]

scaler = StandardScaler()
X = df_model[INDEPENDENT_VARS].copy()
X[continuous_vars] = scaler.fit_transform(X[continuous_vars])

print(f"  ✓ Standardized {len(continuous_vars)} continuous variables")

# Store all results
all_results = {}


# ============================================================================
# STEP 5: MODEL TRAINING - Loop through dependent variables
# ============================================================================

print("\n" + "=" * 80)
print("[STEP 5] Model Training & Evaluation / 模型训练与评估")
print("=" * 80)

for dep_var, var_type in DEPENDENT_VARS.items():
    
    print(f"\n\n{'='*80}")
    print(f"  DEPENDENT VARIABLE: {dep_var} ({var_type})")
    print(f"{'='*80}")
    
    y = df_model[dep_var].values
    X_data = X.values
    
    # Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_data, y, test_size=0.2, random_state=42, 
        stratify=y if var_type == 'binary' else None
    )
    
    print(f"\n  Train: {len(X_train):,}, Test: {len(X_test):,}")
    
    results = []
    models_dict = {}
    predictions_dict = {}
    
    # =========================================================================
    # CONTINUOUS VARIABLE MODELS
    # =========================================================================
    if var_type == 'continuous':
        
        # Model A: OLS
        print(f"\n  --- Model A: OLS Regression ---")
        X_train_sm = sm.add_constant(X_train)
        X_test_sm = sm.add_constant(X_test)
        
        ols_model = sm.OLS(y_train, X_train_sm).fit()
        y_pred_ols = ols_model.predict(X_test_sm)
        
        mae = mean_absolute_error(y_test, y_pred_ols)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_ols))
        r2 = r2_score(y_test, y_pred_ols)
        
        results.append({'Model': 'OLS', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        models_dict['OLS'] = ols_model
        predictions_dict['OLS'] = y_pred_ols
        
        print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
        
        # Save OLS summary
        with open(f'{OUTPUT_DIR}/ols_summary_{dep_var}.txt', 'w') as f:
            f.write(ols_model.summary().as_text())
        
        # Model B: Gradient Boosting
        print(f"\n  --- Model B: Gradient Boosting ---")
        gbdt = GradientBoostingRegressor(n_estimators=200, max_depth=5, 
                                          learning_rate=0.1, random_state=42)
        gbdt.fit(X_train, y_train)
        y_pred_gbdt = gbdt.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred_gbdt)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_gbdt))
        r2 = r2_score(y_test, y_pred_gbdt)
        
        results.append({'Model': 'GBDT', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        models_dict['GBDT'] = gbdt
        predictions_dict['GBDT'] = y_pred_gbdt
        
        print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
        
        # Model C: Random Forest
        print(f"\n  --- Model C: Random Forest ---")
        rf = RandomForestRegressor(n_estimators=200, max_depth=10, 
                                    min_samples_split=10, random_state=42, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred_rf = rf.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred_rf)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
        r2 = r2_score(y_test, y_pred_rf)
        
        results.append({'Model': 'Random Forest', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        models_dict['RF'] = rf
        predictions_dict['RF'] = y_pred_rf
        
        print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
    
    # =========================================================================
    # COUNT VARIABLE MODELS
    # =========================================================================
    elif var_type == 'count':
        
        # Model A: OLS (as baseline)
        print(f"\n  --- Model A: OLS Regression ---")
        X_train_sm = sm.add_constant(X_train)
        X_test_sm = sm.add_constant(X_test)
        
        ols_model = sm.OLS(y_train, X_train_sm).fit()
        y_pred_ols = ols_model.predict(X_test_sm)
        
        mae = mean_absolute_error(y_test, y_pred_ols)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_ols))
        r2 = r2_score(y_test, y_pred_ols)
        
        results.append({'Model': 'OLS', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        models_dict['OLS'] = ols_model
        predictions_dict['OLS'] = y_pred_ols
        
        print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
        
        with open(f'{OUTPUT_DIR}/ols_summary_{dep_var}.txt', 'w') as f:
            f.write(ols_model.summary().as_text())
        
        # Model B: Poisson Regression
        print(f"\n  --- Model B: Poisson Regression ---")
        try:
            poisson = PoissonRegressor(alpha=0.1, max_iter=1000)
            poisson.fit(X_train, y_train)
            y_pred_poisson = poisson.predict(X_test)
            
            mae = mean_absolute_error(y_test, y_pred_poisson)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred_poisson))
            r2 = r2_score(y_test, y_pred_poisson)
            
            results.append({'Model': 'Poisson', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
            models_dict['Poisson'] = poisson
            predictions_dict['Poisson'] = y_pred_poisson
            
            print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
        except Exception as e:
            print(f"    ⚠ Poisson failed: {e}")
        
        # Model C: Gradient Boosting
        print(f"\n  --- Model C: Gradient Boosting ---")
        gbdt = GradientBoostingRegressor(n_estimators=200, max_depth=5, 
                                          learning_rate=0.1, random_state=42)
        gbdt.fit(X_train, y_train)
        y_pred_gbdt = gbdt.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred_gbdt)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_gbdt))
        r2 = r2_score(y_test, y_pred_gbdt)
        
        results.append({'Model': 'GBDT', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        models_dict['GBDT'] = gbdt
        predictions_dict['GBDT'] = y_pred_gbdt
        
        print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
        
        # Model D: Random Forest
        print(f"\n  --- Model D: Random Forest ---")
        rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred_rf = rf.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred_rf)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
        r2 = r2_score(y_test, y_pred_rf)
        
        results.append({'Model': 'Random Forest', 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        models_dict['RF'] = rf
        predictions_dict['RF'] = y_pred_rf
        
        print(f"    R² = {r2:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}")
    
    # =========================================================================
    # BINARY VARIABLE MODELS
    # =========================================================================
    elif var_type == 'binary':
        
        # Model A: Logistic Regression (statsmodels)
        print(f"\n  --- Model A: Logistic Regression ---")
        X_train_sm = sm.add_constant(X_train)
        X_test_sm = sm.add_constant(X_test)
        
        logit_model = Logit(y_train, X_train_sm).fit(disp=0)
        y_pred_prob_logit = logit_model.predict(X_test_sm)
        y_pred_logit = (y_pred_prob_logit >= 0.5).astype(int)
        
        acc = accuracy_score(y_test, y_pred_logit)
        auc = roc_auc_score(y_test, y_pred_prob_logit)
        f1 = f1_score(y_test, y_pred_logit)
        
        results.append({'Model': 'Logistic', 'Accuracy': acc, 'AUC': auc, 'F1': f1})
        models_dict['Logistic'] = logit_model
        predictions_dict['Logistic'] = y_pred_prob_logit
        
        print(f"    Accuracy = {acc:.4f}, AUC = {auc:.4f}, F1 = {f1:.4f}")
        
        with open(f'{OUTPUT_DIR}/logit_summary_{dep_var}.txt', 'w') as f:
            f.write(logit_model.summary().as_text())
        
        # Model B: Gradient Boosting Classifier
        print(f"\n  --- Model B: Gradient Boosting Classifier ---")
        gbdt_clf = GradientBoostingClassifier(n_estimators=200, max_depth=5, 
                                               learning_rate=0.1, random_state=42)
        gbdt_clf.fit(X_train, y_train)
        y_pred_prob_gbdt = gbdt_clf.predict_proba(X_test)[:, 1]
        y_pred_gbdt = gbdt_clf.predict(X_test)
        
        acc = accuracy_score(y_test, y_pred_gbdt)
        auc = roc_auc_score(y_test, y_pred_prob_gbdt)
        f1 = f1_score(y_test, y_pred_gbdt)
        
        results.append({'Model': 'GBDT', 'Accuracy': acc, 'AUC': auc, 'F1': f1})
        models_dict['GBDT'] = gbdt_clf
        predictions_dict['GBDT'] = y_pred_prob_gbdt
        
        print(f"    Accuracy = {acc:.4f}, AUC = {auc:.4f}, F1 = {f1:.4f}")
        
        # Model C: Random Forest Classifier
        print(f"\n  --- Model C: Random Forest Classifier ---")
        rf_clf = RandomForestClassifier(n_estimators=200, max_depth=10, 
                                         min_samples_split=10, random_state=42, n_jobs=-1)
        rf_clf.fit(X_train, y_train)
        y_pred_prob_rf = rf_clf.predict_proba(X_test)[:, 1]
        y_pred_rf = rf_clf.predict(X_test)
        
        acc = accuracy_score(y_test, y_pred_rf)
        auc = roc_auc_score(y_test, y_pred_prob_rf)
        f1 = f1_score(y_test, y_pred_rf)
        
        results.append({'Model': 'Random Forest', 'Accuracy': acc, 'AUC': auc, 'F1': f1})
        models_dict['RF'] = rf_clf
        predictions_dict['RF'] = y_pred_prob_rf
        
        print(f"    Accuracy = {acc:.4f}, AUC = {auc:.4f}, F1 = {f1:.4f}")
    
    # Store results
    all_results[dep_var] = {
        'type': var_type,
        'results': pd.DataFrame(results),
        'models': models_dict,
        'predictions': predictions_dict,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    }
    
    # =========================================================================
    # VISUALIZATIONS FOR THIS DEPENDENT VARIABLE
    # =========================================================================
    
    results_df = pd.DataFrame(results)
    
    # Model Comparison Chart
    if var_type in ['continuous', 'count']:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        for idx, metric in enumerate(['MAE', 'RMSE', 'R2']):
            ax = axes[idx]
            colors = plt.cm.Set2(np.linspace(0, 1, len(results_df)))
            bars = ax.bar(results_df['Model'], results_df[metric], color=colors, edgecolor='black')
            ax.set_ylabel(metric)
            ax.set_title(f'{metric}')
            ax.tick_params(axis='x', rotation=45)
            for bar, val in zip(bars, results_df[metric]):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                       f'{val:.3f}', ha='center', va='bottom', fontsize=10)
        
        fig.suptitle(f'Model Comparison: {dep_var}', fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/06_model_comparison_{dep_var}.png', dpi=150, bbox_inches='tight')
        plt.close()
        
    else:  # binary
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        for idx, metric in enumerate(['Accuracy', 'AUC', 'F1']):
            ax = axes[idx]
            colors = plt.cm.Set2(np.linspace(0, 1, len(results_df)))
            bars = ax.bar(results_df['Model'], results_df[metric], color=colors, edgecolor='black')
            ax.set_ylabel(metric)
            ax.set_title(f'{metric}')
            ax.set_ylim(0, 1)
            ax.tick_params(axis='x', rotation=45)
            for bar, val in zip(bars, results_df[metric]):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                       f'{val:.3f}', ha='center', va='bottom', fontsize=10)
        
        fig.suptitle(f'Model Comparison: {dep_var}', fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/06_model_comparison_{dep_var}.png', dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"\n  ✓ Saved: 06_model_comparison_{dep_var}.png")
    
    # Feature Importance (Random Forest)
    if 'RF' in models_dict:
        rf_model = models_dict['RF']
        
        importance_df = pd.DataFrame({
            'Variable': INDEPENDENT_VARS,
            'Importance': rf_model.feature_importances_
        }).sort_values('Importance', ascending=True)
        
        colors = ['coral' if v in BE_VARS else 'steelblue' for v in importance_df['Variable']]
        
        fig, ax = plt.subplots(figsize=(12, 10))
        ax.barh(importance_df['Variable'], importance_df['Importance'], color=colors, edgecolor='black')
        ax.set_xlabel('Feature Importance')
        ax.set_title(f'Random Forest Feature Importance: {dep_var}', fontsize=14)
        
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='coral', edgecolor='black', label='Built Environment'),
            Patch(facecolor='steelblue', edgecolor='black', label='Demographics')
        ]
        ax.legend(handles=legend_elements, loc='lower right')
        
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/07_feature_importance_{dep_var}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Saved: 07_feature_importance_{dep_var}.png")
    
    # ROC Curve (for binary)
    if var_type == 'binary':
        fig, ax = plt.subplots(figsize=(10, 8))
        
        for model_name, y_pred_prob in predictions_dict.items():
            fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
            auc = roc_auc_score(y_test, y_pred_prob)
            ax.plot(fpr, tpr, linewidth=2, label=f'{model_name} (AUC = {auc:.3f})')
        
        ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Guess')
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f'ROC Curve: {dep_var}', fontsize=14)
        ax.legend(loc='lower right')
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])
        
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/08_roc_curve_{dep_var}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Saved: 08_roc_curve_{dep_var}.png")
        
        # Confusion Matrix
        fig, axes = plt.subplots(1, len(predictions_dict), figsize=(5*len(predictions_dict), 5))
        if len(predictions_dict) == 1:
            axes = [axes]
        
        for idx, (model_name, y_pred_prob) in enumerate(predictions_dict.items()):
            y_pred = (y_pred_prob >= 0.5).astype(int)
            cm = confusion_matrix(y_test, y_pred)
            
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                       xticklabels=['No Transit', 'Transit'],
                       yticklabels=['No Transit', 'Transit'])
            axes[idx].set_xlabel('Predicted')
            axes[idx].set_ylabel('Actual')
            axes[idx].set_title(f'{model_name}')
        
        fig.suptitle(f'Confusion Matrices: {dep_var}', fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/09_confusion_matrix_{dep_var}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Saved: 09_confusion_matrix_{dep_var}.png")
    
    # Partial Dependence Plots
    if 'RF' in models_dict and var_type != 'binary':
        rf_model = models_dict['RF']
        key_be_indices = [INDEPENDENT_VARS.index(v) for v in ['D1B', 'dist_rail_mi', 'bus_count_14mile', 'dist_cbd_mi']]
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        PartialDependenceDisplay.from_estimator(
            rf_model, X_train, features=key_be_indices,
            feature_names=INDEPENDENT_VARS, ax=axes, kind='average'
        )
        fig.suptitle(f'Partial Dependence Plots: {dep_var}', fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/10_pdp_{dep_var}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Saved: 10_pdp_{dep_var}.png")

In [2]:
# ============================================================================
# STEP 6: SUMMARY COMPARISON
# ============================================================================

print("\n\n" + "=" * 80)
print("[STEP 6] Summary Comparison / 综合比较")
print("=" * 80)

# Combine all results
all_comparison = []
for dep_var, data in all_results.items():
    for _, row in data['results'].iterrows():
        row_dict = row.to_dict()
        row_dict['Dependent Variable'] = dep_var
        row_dict['Type'] = data['type']
        all_comparison.append(row_dict)

comparison_df = pd.DataFrame(all_comparison)
comparison_df.to_csv(f'{OUTPUT_DIR}/model_comparison_all.csv', index=False)

print("\n  All Model Results:")
print(comparison_df.to_string(index=False))

# ============================================================================
# Visualization: Overall Comparison
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Continuous/Count models - R²
ax = axes[0]
subset = comparison_df[comparison_df['Type'].isin(['continuous', 'count'])]
if len(subset) > 0:
    pivot = subset.pivot(index='Dependent Variable', columns='Model', values='R2')
    pivot.plot(kind='bar', ax=ax, edgecolor='black')
    ax.set_ylabel('R² Score')
    ax.set_title('R² Comparison (Continuous/Count Outcomes)')
    ax.legend(title='Model')
    ax.tick_params(axis='x', rotation=45)

# Binary models - AUC
ax = axes[1]
subset = comparison_df[comparison_df['Type'] == 'binary']
if len(subset) > 0:
    pivot = subset.pivot(index='Dependent Variable', columns='Model', values='AUC')
    pivot.plot(kind='bar', ax=ax, edgecolor='black')
    ax.set_ylabel('AUC Score')
    ax.set_title('AUC Comparison (Binary Outcome)')
    ax.legend(title='Model')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/11_overall_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n  ✓ Saved: 11_overall_comparison.png")

# ============================================================================
# Visualization: Coefficient/Odds Ratio Comparison
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(20, 10))

for idx, (dep_var, data) in enumerate(all_results.items()):
    ax = axes[idx]
    
    if data['type'] == 'binary' and 'Logistic' in data['models']:
        model = data['models']['Logistic']
        coef_df = pd.DataFrame({
            'Variable': INDEPENDENT_VARS,
            'Coefficient': model.params[1:],
            'Odds_Ratio': np.exp(model.params[1:]),
            'P-value': model.pvalues[1:]
        })
    else:
        model = data['models'].get('OLS')
        if model:
            coef_df = pd.DataFrame({
                'Variable': INDEPENDENT_VARS,
                'Coefficient': model.params[1:],
                'P-value': model.pvalues[1:]
            })
        else:
            continue
    
    coef_df = coef_df.sort_values('Coefficient', ascending=True)
    colors = ['green' if p < 0.05 else 'gray' for p in coef_df['P-value']]
    
    ax.barh(coef_df['Variable'], coef_df['Coefficient'], color=colors, edgecolor='black')
    ax.axvline(0, color='black', linewidth=1)
    ax.set_xlabel('Coefficient (Standardized)')
    ax.set_title(f'{dep_var}')

fig.suptitle('OLS/Logit Coefficients (Green = p < 0.05)', fontsize=14)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/12_coefficients_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("  ✓ Saved: 12_coefficients_comparison.png")

In [2]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n\n" + "=" * 80)
print("FINAL SUMMARY / 最终总结")
print("=" * 80)

print(f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    TRANSIT ANALYSIS COMPLETE                                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

  Analysis Overview / 分析概览:
  ────────────────────────────────────────────────────────────────────────────
    Sample Size:              {len(df_model):,}
    Dependent Variables:      3 (1 continuous, 1 count, 1 binary)
    Independent Variables:    {len(INDEPENDENT_VARS)}

  Best Models by Outcome / 各因变量最佳模型:
  ────────────────────────────────────────────────────────────────────────────
""")

for dep_var, data in all_results.items():
    results = data['results']
    if data['type'] == 'binary':
        best = results.loc[results['AUC'].idxmax()]
        print(f"    {dep_var}:")
        print(f"      Best: {best['Model']} (AUC = {best['AUC']:.4f})")
    else:
        best = results.loc[results['R2'].idxmax()]
        print(f"    {dep_var}:")
        print(f"      Best: {best['Model']} (R² = {best['R2']:.4f})")

print(f"""
  Output Files / 输出文件:
  ────────────────────────────────────────────────────────────────────────────
    Directory: {OUTPUT_DIR}/
    
    ✓ vif_results.csv
    ✓ model_comparison_all.csv
    ✓ ols_summary_*.txt / logit_summary_*.txt
    ✓ 01-12 visualization PNGs

══════════════════════════════════════════════════════════════════════════════
                         ✓ ANALYSIS COMPLETE
══════════════════════════════════════════════════════════════════════════════
""")

    CHICAGO TRAVEL BEHAVIOR - TRANSIT ANALYSIS
    芝加哥出行行为 - 公交出行分析

Configuration:
  Dependent Variables:
    - Y_transit_duration_min (continuous)
    - n_transit_trips (count)
    - has_transit_travel (binary)
  Demographic Variables: 10
  Built Environment Variables: 10

[STEP 1] Loading Data / 加载数据

  Loaded: 6,129 observations
  Final sample size: 6,129

  Dependent Variable Summary:

    Y_transit_duration_min (continuous):
      Mean: 31.77, Std: 56.61
      Min: 0, Max: 960

    n_transit_trips (count):
      Mean: 0.68, Std: 0.95
      Min: 0, Max: 8

    has_transit_travel (binary):
      Mean: 0.39, Std: 0.49
      Min: 0, Max: 1
      Class 1: 2,392 (39.0%)
      Class 0: 3,737 (61.0%)

[STEP 2] Exploratory Data Analysis / 探索性数据分析

  ✓ Saved: 01_transit_dv_distributions.png
  ✓ Saved: 02_transit_by_demographics.png
  ✓ Saved: 03_transit_by_be.png
  ✓ Saved: 04_correlation_matrix.png

[STEP 3] Multicollinearity Check (VIF) / 多重共线性检验

  Variance Inflation Factors:
  --------